In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [ ]:
#Import wine reviews csv files and read into dataframe
kaggle_wine_file = 'Resources/wine-reviews/winemag-data-130k-v2.csv'
kaggle_wine_df = pd.read_csv(kaggle_wine_file, encoding="ISO-8859-1")
data_world_wine_file = 'Resources/Wines.csv'
data_world_wine_df = pd.read_csv(data_world_wine_file, encoding="ISO-8859-1")

In [ ]:
#Display both dataframes
kaggle_wine_df.head()

In [ ]:
data_world_wine_df.head()

In [ ]:
#Adjust case sensitive columns
kaggle_wine_df = kaggle_wine_df.rename(columns={"country": "Country", "designation": "Designation", "points": "Points", "price": "Price", "province": "Province", "title": "Title", "variety": "Variety", "winery": "Winery"})

In [ ]:
#Merge the two dataframes
merged_wine_df = pd.merge(kaggle_wine_df, data_world_wine_df, on="Title")
#'Designation', 'Points', 'Price', 'Province', 'Title', 'Variety', 'Winery'])
merged_wine_df.head()

In [ ]:
#Drop all null values from the merged dataframe
not_null_wine_df= merged_wine_df.dropna(how='any')
not_null_wine_df.head()

In [ ]:
#Drop extra columns in dataframe
cleaned_wine_df= not_null_wine_df.drop(['Unnamed: 0','description','region_2','taster_twitter_handle', 'Country_y', 'Designation_y', 'Points_y', 'Price_y', 'Province_y', 'Variety_y', 'Winery_y'], axis=1)
cleaned_wine_df.head()

In [ ]:
#Rename columns
cleaned_wine_df = cleaned_wine_df.rename(columns={"Country_x": "country", "Designation_x": "designation", "Points_x": "points", "Price_x": "price", "Province_x": "province", "Variety_x": "variety", "Winery_x": "winery", "Title": "title", "Vintage": "vintage", "County": "county", "region_1": "region"})
cleaned_wine_df.head()

In [ ]:
#Connect to database
wine_connection_string = "postgres:Bruh1need@pwd@localhost:5432/wine_db"
engine = create_engine(f'postgresql://{wine_connection_string}')

In [ ]:
#Check table names
engine.table_names()

In [ ]:
#Convert cleaned dataframe to sql database
cleaned_wine_df.to_sql(name='wine_table', con=engine, if_exists='append', index=False)

In [ ]:
#Confirm that the sql database has been created
pd.read_sql_query('select * from wine_table', con=engine).head()

In [2]:
#Import the JSON file
json_file = "../Resources/winemag-data-130k-v2.json"
wine_json_df = pd.read_json(json_file)
wine_json_df.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,None,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,None,None,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",None,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,None,Alexander Peartree,None,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [3]:
#Create a new table dropping columns we do not want
new_wine_json_df = wine_json_df[["title", "variety","winery", "points", "price"]].copy()
new_wine_json_df.head()

,title,variety,winery,points,price
0,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,87,NaN
1,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,87,15.0
2,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,87,14.0
3,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,87,13.0
4,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,87,65.0


In [4]:
#Clean data
new_wine_json_df = new_wine_json_df.dropna(how='any')
new_wine_json_df.head()

,title,variety,winery,points,price
1,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,87,15.0
2,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,87,14.0
3,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,87,13.0
4,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,87,65.0
5,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,87,15.0


In [6]:
#Bring up only the best wines with the highest score.
new_wine_json_df = new_wine_json_df.loc[new_wine_json_df['points'] == 100]
new_wine_json_df.sort_values(by=['points'], inplace=True, ascending=False)
new_wine_json_df.head()

,title,variety,winery,points,price
345,Chambers Rosewood Vineyards NV Rare Muscat (Ru...,Muscat,Chambers Rosewood Vineyards,100,350.0
7335,Avignonesi 1995 Occhio di Pernice (Vin Santo ...,Prugnolo Gentile,Avignonesi,100,210.0
36528,Krug 2002 Brut (Champagne),Champagne Blend,Krug,100,259.0
39286,Tenuta dell'Ornellaia 2007 Masseto Merlot (Tos...,Merlot,Tenuta dell'Ornellaia,100,460.0
42197,Casa Ferreirinha 2008 Barca-Velha Red (Douro),Portuguese Red,Casa Ferreirinha,100,450.0


In [7]:
#Setup SQL connection
rds_connection_string = "postgres:postgres@localhost:5432/wine_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [8]:
#Check tables name from the SQL connection
engine.table_names()

['wine_json']

In [9]:
#Send the data to Postgres
new_wine_json_df.to_sql(name='wine_json', con=engine, if_exists='append', index=False)

In [10]:
#Query the database to make sure the data made it
pd.read_sql_query('select * from wine_json', con=engine).head()

,id,title,variety,winery,points,price
0,1,Chambers Rosewood Vineyards NV Rare Muscat (Ru...,Muscat,Chambers Rosewood Vineyards,100,350.0
1,2,Avignonesi 1995 Occhio di Pernice (Vin Santo ...,Prugnolo Gentile,Avignonesi,100,210.0
2,3,Krug 2002 Brut (Champagne),Champagne Blend,Krug,100,259.0
3,4,Tenuta dell'Ornellaia 2007 Masseto Merlot (Tos...,Merlot,Tenuta dell'Ornellaia,100,460.0
4,5,Casa Ferreirinha 2008 Barca-Velha Red (Douro),Portuguese Red,Casa Ferreirinha,100,450.0
